In [ ]:
import os, glob
import pandas as pd
from mmengine.config import Config
from mmengine.runner import Runner
from mmdet.registry import DATASETS
from mmengine.registry import DefaultScope

# ========= User paths =========
CONFIG ='C:/Users/heheh/mmdetection/configs/mask_rcnn/mask-rcnn_r50_fpn_1x_coco.py'
CHECKPOINT = 'C:/Users/heheh/mmdetection/checkpoints/mask_rcnn_r50_fpn_1x_coco_20200205-d4b0c5d6.pth'
DATA_ROOT = 'C:/Users/heheh/mmdetection/data/coco'
ANN_FILE  = os.path.join(DATA_ROOT, 'annotations', 'instances_val2017.json')
CLEAN_IMG_DIR = os.path.join(DATA_ROOT, 'val2017')

# Add any number of adversarial dirs here (only those that exist will be used)
ADV_DIRS = {
    'deeplabv3_pgd': r'C:/Users/heheh/mmsegmentation/data/semantic_adv_deeplabv3',
    'maskrcnn_pgd':  r'C:/Users/heheh/mmdetection/data/coco/instance_maskrcnn_adv',
    'panoptic_pgd':  r'C:/Users/heheh/mmdetection/data/coco/panoptic_fpn_adv',
}

OUT_DIR = './work_dirs/eval_multi'
CSV_PATH = os.path.join(OUT_DIR, 'maskrcnn_clean_vs_adversarials.csv')
os.makedirs(OUT_DIR, exist_ok=True)

# ========= Helpers =========
def ensure_mmdet_scope():
    # Set/override the default registry scope so TRANSFORMS resolve to mmdet.*
    try:
        # If a scope is already active
        DefaultScope.get_current_instance().set_scope('mmdet')
    except Exception:
        # If no scope yet, create one
        try:
            DefaultScope.get_instance('eval-scope', scope_name='mmdet')
        except Exception:
            pass
def make_cfg(img_prefix, indices, work_dir):
    cfg = Config.fromfile(CONFIG)
    cfg.default_scope = 'mmdet'
    cfg.load_from = CHECKPOINT
    cfg.work_dir = work_dir
    if hasattr(cfg, 'visualizer'):
        cfg.visualizer.vis_backends = None
    # dataloader
    cfg.test_dataloader.batch_size = 2
    cfg.test_dataloader.num_workers = 2
    ds = cfg.test_dataloader.dataset
    ds.type = 'CocoDataset'
    ds.data_root = DATA_ROOT
    ds.ann_file = ANN_FILE
    ds.data_prefix = dict(img=img_prefix)
    ds.test_mode = True
    ds.indices = indices
    # evaluator
    cfg.test_evaluator = dict(
        type='CocoMetric',
        ann_file=ANN_FILE,
        metric=['bbox', 'segm'],
        format_only=False
    )
    return cfg

def build_dataset_for_mapping(img_dir):
    """Build a CocoDataset once to map dataset index -> real img path."""
    tmp_cfg = make_cfg(img_dir, indices=None, work_dir=os.path.join(OUT_DIR, 'tmp'))
    ensure_mmdet_scope() 
    return DATASETS.build(tmp_cfg.test_dataloader.dataset)

def basenames_in_dir(d):
    if not os.path.isdir(d):
        return set()
    exts = ('.jpg','.jpeg','.png')
    return {os.path.splitext(os.path.basename(p))[0]
            for p in glob.glob(os.path.join(d, '*'))
            if p.lower().endswith(exts)}

def compute_intersection_indices(clean_ds, dir_list):
    """
    Build subset indices using the intersection of basenames that exist
    in *all* provided directories and in the clean dataset.
    """
    # map clean dataset base -> dataset index
    base_to_idx = {}
    for i in range(len(clean_ds)):
        info = clean_ds.get_data_info(i)
        base = os.path.splitext(os.path.basename(info['img_path']))[0]
        base_to_idx[base] = i

    # intersection of basenames across all dirs
    sets = []
    for d in dir_list:
        s = basenames_in_dir(d)
        if not s:
            print(f'[WARN] Empty or missing dir skipped in intersection: {d}')
        sets.append(s)
    # Always include clean basenames (so indices are valid)
    sets.append(set(base_to_idx.keys()))

    common = set.intersection(*sets) if sets else set()
    indices = sorted([base_to_idx[b] for b in common if b in base_to_idx])
    return indices, common

def eval_one(label, img_dir, indices):
    cfg = make_cfg(img_dir, indices, work_dir=os.path.join(OUT_DIR, f'eval_{label}'))
    runner = Runner.from_cfg(cfg)
    metrics = runner.test()  # dict
    # Flatten to simple dict with label
    row = {'set': label, 'count': len(indices)}
    for k, v in metrics.items():
        row[k] = v
    return row

def add_drop_columns(df, clean_row_name='clean'):
    """Add drop (adv - clean) and drop% columns vs clean for key metrics."""
    # pivot: set rows indexed by 'set'
    df = df.set_index('set')
    if clean_row_name not in df.index:
        raise RuntimeError('Clean results missing; cannot compute drops.')
    clean = df.loc[clean_row_name]

    keys = [
        'coco/bbox_mAP','coco/bbox_mAP_50','coco/bbox_mAP_75',
        'coco/segm_mAP','coco/segm_mAP_50','coco/segm_mAP_75'
    ]
    for k in keys:
        col_drop = f'{k}_drop'
        col_drop_pct = f'{k}_drop_pct'
        df[col_drop] = df[k] - float(clean[k])
        # relative drop % (negative means a drop)
        df[col_drop_pct] = 100.0 * (df[k] - float(clean[k])) / max(1e-8, float(clean[k]))

    # "Success rate" = relative drop at AP50 (bounded to [0,1])
    for k, name in [('coco/bbox_mAP_50','success_rate_bbox'),
                    ('coco/segm_mAP_50','success_rate_segm')]:
        sr = 1.0 - (df[k] / max(1e-8, float(clean[k])))
        df[name] = sr.clip(lower=0.0, upper=1.0)

    # move clean to top again and reset index for CSV
    return df.reset_index()

# ========= Run =========
# Build clean dataset mapping
clean_ds = build_dataset_for_mapping(CLEAN_IMG_DIR)

# Keep only adversarial dirs that actually exist
adv_dirs_exist = {k:v for k,v in ADV_DIRS.items() if os.path.isdir(v)}
if not adv_dirs_exist:
    raise RuntimeError('No adversarial directories found. Check ADV_DIRS paths.')

# Build a *common* subset across all adv dirs (and clean)
indices, common_bases = compute_intersection_indices(clean_ds, list(adv_dirs_exist.values()))
print(f'[INFO] Common subset size (intersection across adv dirs): {len(indices)}')
# --- Limit to 10 images ---
N = 5000                    # how many images to test
SAMPLE_MODE = 'first'      # 'first' or 'random'
RANDOM_SEED = 123

if len(indices) > N:
    if SAMPLE_MODE == 'random':
        import random
        random.seed(RANDOM_SEED)
        indices = sorted(random.sample(indices, N))
    else:
        indices = indices[:N]

print(f'[INFO] Using {len(indices)} images for evaluation (mode={SAMPLE_MODE}).')

# Evaluate clean on that subset
rows = []
rows.append(eval_one('clean', CLEAN_IMG_DIR, indices))

# Evaluate each adv dir on the same subset
for label, img_dir in adv_dirs_exist.items():
    print(f'[INFO] Evaluating {label} on {len(indices)} images …')
    rows.append(eval_one(label, img_dir, indices))

# Build table, add drops/success-rate, save CSV
df = pd.DataFrame(rows)
df = add_drop_columns(df, clean_row_name='clean')
print('\n===== Summary (clean vs adversarials) =====')
print(df.to_string(index=False))

df.to_csv(CSV_PATH, index=False)
print(f'\n[OK] Saved CSV → {CSV_PATH}')
